In [ ]:
import io.jhdf.HdfFile
import java.nio.file.Path

%use dataframe, kandy

Open the file and read the weather data into memory

In [ ]:
val file = HdfFile(Path.of("/home/james/git/jhdf-webinar/hdf5-example/weather.hdf5"))
val weatherDataset = file.getDatasetByPath("weather/table")
val compountData = weatherDataset.getData() as Map<String, Any>

Convert the java map into a data frame

In [ ]:
val country = columnOf(*(compountData.get("country") as Array<String>)).named("country")
val date = columnOf(*(compountData.get("date") as Array<String>)).named("date")
val temp = columnOf(*(compountData.get("avg_temp_c") as DoubleArray).toTypedArray()).named("temp")
val rainfall = columnOf(*((compountData.get("precipitation_mm") as DoubleArray).toTypedArray())).named("rainfall")
val pressure = columnOf(*((compountData.get("avg_sea_level_pres_hpa")  as DoubleArray).toTypedArray())).named("pressure")
val df = dataFrameOf(country, date, temp, rainfall, pressure)
    .parse { date }

See the countries available

In [ ]:
df.country.distinct().toList()

Plot the data for a few countries

In [ ]:
val countries = setOf("Spain", "Finland", "United Kingdom", "United States")
df
    .filter { country in countries }
    .filter { temp.isFinite() }
    .filter { date in LocalDate.parse("1980-01-01")..LocalDate.parse("2000-01-01") }
    .groupBy(country).plot { 
        line { 
            x(date)
            y(temp)
            color(country)
        }
     }